In [1]:
%load_ext autoreload
%autoreload 2
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch_geometric.datasets.qm9 import QM9
from torch_geometric.data import DataLoader
import torch_geometric.nn as tgnn 
import tqdm
import numpy as np

In [2]:
# [0] Reports MAE in eV / Chemical Accuracy of the target variable U0. 
# The chemical accuracy of U0 is 0.043 see [1, Table 5].

# Reproduced table [0]
# MXMNet: 0.00590/0.043 = 0.13720930232558143
# HMGNN:  0.00592/0.043 = 0.13767441860465118
# MPNN:   0.01935/0.043 = 0.45
# KRR:    0.0251 /0.043 = 0.5837209302325582
# [0] https://paperswithcode.com/sota/formation-energy-on-qm9
# [1] Neural Message Passing for Quantum Chemistry, https://arxiv.org/pdf/1704.01212v2.pdf
# MXMNet https://arxiv.org/pdf/2011.07457v1.pdf
# HMGNN https://arxiv.org/pdf/2009.12710v1.pdf
# MPNN https://arxiv.org/pdf/1704.01212v2.pdf
# KRR HDAD kernel ridge regression https://arxiv.org/pdf/1702.05532.pdf
# HDAD means HDAD (Histogram of distances, anglesand dihedral angles)

# [2] Reports the average value of MAE / Chemical Accuracy of over all targets
# [2] https://paperswithcode.com/sota/drug-discovery-on-qm9
target_dict = {0: 'mu, D, Dipole moment', 
               1: 'alpha, {a_0}^3, Isotropic polarizability', 
               2: 'epsilon_{HOMO}, eV, Highest occupied molecular orbital energy',
               3: 'epsilon_{LUMO}, eV, Lowest unoccupied molecular orbital energy',
               4: 'Delta, eV, Gap between HOMO and LUMO',
               5: '< R^2 >, {a_0}^2, Electronic spatial extent',
               6: 'ZPVE, eV, Zero point vibrational energy', 
               7: 'U_0, eV, Internal energy at 0K',
               8: 'U, eV, Internal energy at 298.15K', 
               9: 'H, eV, Enthalpy at 298.15K',
               10: 'G, eV, Free energy at 298.15K',  
               11: 'c_{v}, cal\(mol K), Heat capacity at 298.15K'}

chemical_accuracy = {idx:0.043 for idx in range(12)}
chemical_accuracy[0] = 0.1
chemical_accuracy[1] = 0.1
chemical_accuracy[5] = 1.2
chemical_accuracy[6] = 0.0012
chemical_accuracy[11] = 0.050

# get rid of the degenerate molecules

In [3]:
from urllib import request
import tempfile
import os
at_url = "https://ndownloader.figshare.com/files/3195404"
tmpdir = tempfile.mkdtemp("gdb9")
tmp_path = os.path.join(tmpdir, "uncharacterized.txt")
request.urlretrieve(at_url, tmp_path)

evilmols = []
with open(tmp_path) as f:
    lines = f.readlines()
    for line in lines[9:-1]:
        evilmols.append(int(line.split()[0]))
evilgdbs = ['gdb_%d'%id for id in evilmols]

In [4]:
pre_filter = lambda d: (d.name not in evilgdbs)

In [5]:
dataset = QM9('../datasets/qm9_geometric/', pre_filter=pre_filter)#, pre_filter=pre_filter)

In [6]:
# actually QM9 already automatically gets rid of all the gad examples -.-

In [7]:
dataset = dataset.shuffle()
train_dataset = dataset[:110000]
valid_dataset = dataset[110000:120000]
test_dataset = dataset[120000:]
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True, pin_memory=True)
valid_loader = DataLoader(valid_dataset, batch_size=128)
test_loader = DataLoader(test_dataset, batch_size=32)

In [8]:
batch = next(iter(train_loader))

In [9]:
# investigate batch

In [10]:
batch['batch']

tensor([  0,   0,   0,  ..., 127, 127, 127])

In [11]:
(batch['batch'].detach().numpy() == 2).sum()

20

In [12]:
# node features:

In [13]:
batch.x # one_hot(type), atomic_number, aromatic, sp1, sp2, sp3, num_hs -> 5+1+1+1+1+1+1 = 11

tensor([[0., 0., 1.,  ..., 0., 0., 2.],
        [0., 1., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [1., 0., 0.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.]])

In [14]:
batch.x.shape

torch.Size([2300, 11])

In [15]:
batch.pos

tensor([[-0.1204,  0.4536,  1.2954],
        [-0.0762, -0.4728,  0.2973],
        [ 0.9001, -1.1023, -0.0309],
        ...,
        [-1.1957, -2.8606, -3.0300],
        [-1.5937, -1.1596, -2.7099],
        [-0.1183, -1.5790, -3.6121]])

In [16]:
batch.z # atomic number

tensor([7, 6, 8,  ..., 1, 1, 1])

In [17]:
# edge features:

In [18]:
batch.edge_index

tensor([[   0,    0,    0,  ..., 2297, 2298, 2299],
        [   1,    9,   10,  ..., 2285, 2285, 2285]])

In [19]:
batch.edge_attr # one_hot(bond_type) -> 4 single, double, triple, aromatic

tensor([[1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        ...,
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.]])

In [20]:
# targets 
batch.y[:, 7] # this target is U0

tensor([-12925.9668, -12012.0059, -10968.1504, -11419.2139, -11914.2793,
        -12989.7871, -11543.5000,  -8662.4619, -13396.7100, -10598.3105,
        -10532.9893, -10969.1758, -11980.8047,  -9554.7256,  -9959.5352,
        -11343.0527, -10497.8975, -10938.0781, -10565.4678, -11948.2764,
        -11914.1689, -11373.4238, -11949.8447, -10565.2734,  -8897.6631,
         -8394.1113, -11844.7822, -12554.5693, -11812.5830, -11341.5605,
        -11810.2422, -12894.3945, -11915.2100, -10397.1191, -11979.9766,
        -11373.9668, -11510.3711, -11950.0342, -11913.8320, -12248.1006,
        -10533.2607, -10969.8701, -10848.2070, -12925.5713, -12014.2988,
        -10970.6025, -10565.4189, -14089.6182, -10463.6846, -11316.7227,
        -11543.0498, -10397.0127,  -9958.4756, -11543.1621, -10532.8047,
        -11376.8887, -11003.6660, -12353.6572, -12520.3926,  -9992.2549,
         -9497.5303, -10271.1182, -11303.9033, -10934.8936, -10408.3828,
         -9898.0400, -10500.0322, -11947.6855, -112

In [21]:
ngpu=1
device = torch.device('cuda:0' if (torch.cuda.is_available() and ngpu > 0) else 'cpu')

# Model

In [22]:
model = tgnn.SchNet().to(device)

In [23]:
lr = 0.003
n_epochs = 10000
patience = 1
factor = 0.96
target_idx = 7

In [24]:
target_vec = [d.y[0, target_idx] for d in train_dataset]
target_mean = np.mean(target_vec)
target_std = np.std(target_vec)

In [25]:
def normalize(target, mean=target_mean, std=target_std):
    return (target - mean)/std

def denormalize(target, mean=target_mean, std=target_std):
    return target*std + mean

In [26]:
loss = nn.L1Loss()
optimizer = optim.Adam(model.parameters(), lr=lr)
lr_scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', verbose=True, patience=patience, factor=factor)

# Training

In [ ]:
loss_hist = []
try:
    # For each epoch
    for epoch in range(n_epochs):
        # For each batch in the dataloader
        pbar = tqdm.tqdm(enumerate(train_loader, 0))
        epoch_loss = 0
        for i, data in pbar:
            model.zero_grad()
            data.to(device)
            target = data.y[:,target_idx]
            prediction = model(data.z, data.x, data.batch)
            output = loss(prediction.view(-1), normalize(target))
            mae = (denormalize(prediction.view(-1)) - target).abs().mean()
            epoch_loss = (epoch_loss*i + mae.item())/(i+1)
            
            pbar.set_description('MAE/CA %2.6f'%(epoch_loss/chemical_accuracy[target_idx]))
            output.backward()
            optimizer.step()

        lr_scheduler.step(epoch_loss)
        loss_hist += [epoch_loss]
        if epoch_loss/chemical_accuracy[target_idx] < 0.45:
            break

except KeyboardInterrupt:
    print('keyboard interrupt caught')

MAE/CA 28336.403778: : 860it [00:28, 29.68it/s] 
MAE/CA 7550.289029: : 860it [00:29, 29.48it/s]
MAE/CA 4231.139139: : 860it [00:29, 29.59it/s]
MAE/CA 4252.439653: : 860it [00:29, 29.28it/s]
MAE/CA 4047.639742: : 860it [00:29, 29.47it/s]
MAE/CA 3747.478561: : 860it [00:29, 28.92it/s]
MAE/CA 3427.646766: : 860it [00:29, 29.65it/s]
MAE/CA 3336.948044: : 860it [00:28, 29.67it/s]
MAE/CA 2921.808570: : 860it [00:28, 29.83it/s]
MAE/CA 1967.651528: : 860it [00:28, 29.76it/s]
MAE/CA 765564918526.571533: : 860it [00:29, 29.60it/s]
MAE/CA 19614.992807: : 860it [00:28, 29.95it/s]
MAE/CA 20166.253851: : 4it [00:00, 30.55it/s]

Epoch    12: reducing learning rate of group 0 to 2.8800e-03.


MAE/CA 19623.011282: : 860it [00:28, 29.68it/s]
MAE/CA 19640.395351: : 860it [00:29, 29.48it/s]
MAE/CA 19255.682231: : 3it [00:00, 29.69it/s]

Epoch    14: reducing learning rate of group 0 to 2.7648e-03.


MAE/CA 19669.427893: : 860it [00:28, 30.60it/s]
MAE/CA 19661.641807: : 860it [00:28, 30.01it/s]
MAE/CA 19422.927738: : 3it [00:00, 29.51it/s]

...

MAE/CA 19575.354022: : 860it [00:28, 30.05it/s]
MAE/CA 19574.459439: : 860it [00:28, 30.02it/s]
MAE/CA 19575.140891: : 860it [00:28, 29.85it/s]
MAE/CA 19575.492957: : 860it [00:28, 29.68it/s]
MAE/CA 19580.732173: : 860it [00:28, 29.67it/s]
MAE/CA 19574.942801: : 860it [00:28, 29.95it/s]
MAE/CA 19575.865182: : 860it [00:29, 29.46it/s]
MAE/CA 19574.556114: : 860it [00:29, 29.64it/s]
MAE/CA 19575.197608: : 860it [00:28, 30.48it/s]
MAE/CA 19578.326499: : 860it [00:28, 30.49it/s]
MAE/CA 19573.578361: : 860it [00:29, 29.62it/s]
MAE/CA 19573.446266: : 860it [00:28, 29.76it/s]
MAE/CA 19575.847948: : 860it [00:29, 29.62it/s]
MAE/CA 19576.968470: : 860it [00:28, 29.83it/s]
MAE/CA 19575.289384: : 860it [00:28, 29.76it/s]
MAE/CA 19576.838373: : 860it [00:28, 29.67it/s]
MAE/CA 19577.917152: : 860it [00:28, 30.10it/s]
MAE/CA 19576.266574: : 860it [00:29, 29.54it/s]
MAE/CA 19573.721634: : 860it [00:29, 29.54it/s]
MAE/CA 19574.687851: : 860it [00:28, 29.92it/s]
MAE/CA 19573.476833: : 860it [00:28, 30.

MAE/CA 19576.550678: : 860it [00:28, 29.92it/s]
MAE/CA 19575.043474: : 860it [00:28, 30.23it/s]
MAE/CA 19579.001871: : 860it [00:29, 29.61it/s]
MAE/CA 19576.037957: : 860it [00:28, 29.66it/s]
MAE/CA 19576.692877: : 860it [00:29, 29.61it/s]
MAE/CA 19574.576520: : 860it [00:28, 29.90it/s]
MAE/CA 19577.354138: : 860it [00:28, 29.74it/s]
MAE/CA 19578.884602: : 860it [00:28, 29.72it/s]
MAE/CA 19577.258474: : 860it [00:28, 29.71it/s]
MAE/CA 19575.572753: : 860it [00:28, 29.91it/s]
MAE/CA 19578.324521: : 860it [00:29, 29.55it/s]
MAE/CA 19576.988899: : 860it [00:28, 29.80it/s]
MAE/CA 19578.848357: : 860it [00:29, 29.54it/s]
MAE/CA 19578.762129: : 860it [00:29, 29.43it/s]
MAE/CA 19573.266493: : 860it [00:28, 29.72it/s]
MAE/CA 19577.848008: : 860it [00:28, 30.38it/s]
MAE/CA 19577.174454: : 860it [00:28, 29.67it/s]
MAE/CA 19574.322876: : 860it [00:28, 29.66it/s]
MAE/CA 19575.505496: : 860it [00:28, 30.36it/s]
MAE/CA 19574.793928: : 860it [00:28, 29.83it/s]
MAE/CA 19575.180702: : 860it [00:28, 30.

MAE/CA 19574.453206: : 860it [00:29, 29.64it/s]
MAE/CA 19575.575409: : 860it [00:28, 29.84it/s]
MAE/CA 19578.835361: : 860it [00:28, 29.91it/s]
MAE/CA 19575.095974: : 860it [00:28, 29.81it/s]
MAE/CA 19575.876738: : 860it [00:28, 29.73it/s]
MAE/CA 19578.212658: : 860it [00:28, 30.03it/s]
MAE/CA 19575.687152: : 860it [00:28, 29.74it/s]
MAE/CA 19577.270103: : 860it [00:28, 29.81it/s]
MAE/CA 19576.004515: : 860it [00:28, 29.79it/s]
MAE/CA 19575.354925: : 860it [00:29, 29.64it/s]
MAE/CA 19577.661831: : 860it [00:29, 29.52it/s]
MAE/CA 19576.135044: : 860it [00:28, 29.69it/s]
MAE/CA 19580.324127: : 860it [00:28, 30.30it/s]
MAE/CA 19574.886374: : 860it [00:29, 29.38it/s]
MAE/CA 19576.598824: : 860it [00:29, 29.64it/s]
MAE/CA 19577.300104: : 860it [00:28, 30.47it/s]
MAE/CA 19582.128591: : 860it [00:28, 29.70it/s]
MAE/CA 19576.204553: : 860it [00:28, 29.97it/s]
MAE/CA 19577.609101: : 860it [00:29, 29.63it/s]
MAE/CA 19574.823069: : 860it [00:28, 29.96it/s]
MAE/CA 19571.585850: : 860it [00:28, 29.

MAE/CA 19574.265315: : 860it [00:28, 30.33it/s]
MAE/CA 19575.787142: : 860it [00:28, 30.02it/s]
MAE/CA 19576.301635: : 860it [00:28, 29.67it/s]
MAE/CA 19574.569276: : 860it [00:28, 30.26it/s]
MAE/CA 19574.678451: : 860it [00:28, 29.82it/s]
MAE/CA 19574.433149: : 860it [00:29, 29.38it/s]
MAE/CA 19577.113054: : 860it [00:28, 29.76it/s]
MAE/CA 19577.013792: : 860it [00:28, 30.13it/s]
MAE/CA 19577.092019: : 860it [00:28, 29.85it/s]
MAE/CA 19576.019573: : 860it [00:29, 29.64it/s]
MAE/CA 19576.343303: : 860it [00:29, 29.58it/s]
MAE/CA 19573.223979: : 860it [00:28, 30.08it/s]
MAE/CA 19577.729125: : 860it [00:28, 29.92it/s]
MAE/CA 19576.788266: : 860it [00:28, 30.10it/s]
MAE/CA 19573.368836: : 860it [00:28, 29.79it/s]
MAE/CA 19575.352413: : 860it [00:28, 29.68it/s]
MAE/CA 19575.381433: : 860it [00:29, 29.59it/s]
MAE/CA 19573.753168: : 860it [00:27, 30.84it/s]
MAE/CA 19573.612500: : 860it [00:28, 29.86it/s]
MAE/CA 19575.157013: : 860it [00:28, 29.77it/s]
MAE/CA 19575.956288: : 860it [00:28, 29.

MAE/CA 19575.551318: : 860it [00:29, 29.62it/s]
MAE/CA 19576.197177: : 860it [00:28, 29.89it/s]
MAE/CA 19572.874706: : 860it [00:28, 30.07it/s]
MAE/CA 19577.441007: : 860it [00:28, 29.67it/s]
MAE/CA 19580.333368: : 860it [00:28, 29.75it/s]
MAE/CA 19576.584173: : 860it [00:28, 29.88it/s]
MAE/CA 19575.632386: : 860it [00:28, 30.21it/s]
MAE/CA 19574.549749: : 860it [00:28, 29.90it/s]
MAE/CA 19575.038352: : 860it [00:29, 29.55it/s]
MAE/CA 19574.663663: : 860it [00:29, 29.64it/s]
MAE/CA 19574.803082: : 860it [00:28, 30.05it/s]
MAE/CA 19573.635653: : 860it [00:28, 29.78it/s]
MAE/CA 19575.154867: : 860it [00:28, 29.96it/s]
MAE/CA 19574.615242: : 860it [00:29, 29.65it/s]
MAE/CA 19575.481560: : 860it [00:28, 30.10it/s]
MAE/CA 19577.045498: : 860it [00:29, 29.45it/s]
MAE/CA 19574.419935: : 860it [00:28, 29.98it/s]
MAE/CA 19575.004220: : 860it [00:28, 30.62it/s]
MAE/CA 19574.321571: : 860it [00:28, 30.24it/s]
MAE/CA 19572.583850: : 860it [00:28, 30.20it/s]
MAE/CA 19576.692640: : 860it [00:28, 29.

In [ ]:
model = model.to('cpu')
pbar = tqdm.tqdm(enumerate(test_loader, 0))
epoch_loss = 0
maes = []
for i, data in pbar:
    data.to('cpu')
    prediction = model(data.z, data.x, data.batch)
    mae = (denormalize(prediction.view(-1)) - data.y[:, target_idx]).abs()
    maes += [mae]
maes = torch.cat(maes, dim=0)
mae = maes.mean().item()
print(mae, mae/chemical_accuracy[target_idx])
